---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    """import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 45))
    
    for i,G in enumerate(P1_Graphs) :
        i+=1
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        hist = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
        
        ave_shortest_path_length = nx.average_shortest_path_length(G)
        ave_clustering = nx.average_clustering(G)
        
        ax = plt.subplot(5,1,i)
        ax.plot(degree_values,hist,'o')
        ax.set_xlabel('Degree')
        ax.set_ylabel('Fraction of nodes')
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_title("Graph {} \n ave.shortest path lenghth : {} \n ave.clustering : {}".format(i,ave_shortest_path_length,ave_clustering))
        
    plt.show()"""
    
    return ['PA','SW_L','SW_L','PA','SW_H']
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    
    #l = list(G.nodes(data='ManagementSalary'))
    #nodes , ManagementSalary  = zip(*l)
    
    l = list(G.nodes(data=True))
    ManagementSalary = [x[1]['ManagementSalary'] for x in l]
    nodes = [x[0] for x in l]
    Department = [x[1]['Department'] for x in l]
    degrees = list(G.degree().values())
    betweenness = list(nx.betweenness_centrality(G, normalized=True, endpoints=False).values())
    
    closeness = list(nx.closeness_centrality(G).values())
    
    #d = nx.get_node_attributes(G,'ManagementSalary')
    #ManagementSalary = list(d.values())
    #nodes = list(d.keys())
    
    df = pd.DataFrame({'node': nodes, 'ManagementSalary': ManagementSalary,'Betweenness':betweenness,'Degree':degrees,'Closeness':closeness,'Department' : Department})
    df = df.set_index('node')
    
    df_train = df[df.ManagementSalary.notnull()]
    df_test = df[df.ManagementSalary.isnull()]
    
    #from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    #from sklearn.model_selection import cross_val_score
    
    #clf = LogisticRegression(random_state=0).fit(df_train.iloc[:,:-1], list(df_train.ManagementSalary))
    rfc = RandomForestClassifier(n_estimators=100, max_depth=4).fit(df_train.iloc[:,:-1], list(df_train.ManagementSalary))
    prediction_prob = rfc.predict_proba(df_test.iloc[:,:-1])
    prediction = rfc.predict(df_test.iloc[:,:-1])
    #print("AUC" ,np.mean(cross_val_score(clf, df_train.iloc[:,:-1], list(df_train.ManagementSalary), scoring='roc_auc')))
        
    return pd.DataFrame({'prob_c_1':prediction_prob[:,1]} , index = df_test.index)

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [7]:
def new_connections_predictions():
    
    import operator
    df = future_connections.copy()
    df['Common_neighbours'] = [len(list(nx.common_neighbors(G, e[0], e[1]))) for e in df.index]
    df['jaccard_coefficient'] = [list(nx.jaccard_coefficient(G,[e]))[0][2] for e in df.index]
    df['resource_allocation_index'] = [list(nx.resource_allocation_index(G,[e]))[0][2] for e in df.index]
    df['nx.adamic_adar_index'] = [list(nx.adamic_adar_index(G,[e]))[0][2] for e in df.index]
    df['nx.preferential_attachment'] = [list(nx.preferential_attachment(G,[e]))[0][2] for e in df.index]
    
    
    df_train = df[df["Future Connection"].notnull()]
    df_test = df[df["Future Connection"].isnull()]
    
    from sklearn.linear_model import LogisticRegression
    
    clf = LogisticRegression(random_state=0)
    clf = clf.fit(df_train.drop(['Future Connection'], axis=1), list(df_train["Future Connection"]))
    prediction_prob = clf.predict_proba(df_test.drop(['Future Connection'], axis=1))
    
    
    return pd.DataFrame({'prob_c_1':prediction_prob[:,1]} , index = df_test.index)